# Table of Contents
* [Build dataset](#Build-dataset)


In [1]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [346]:
import glob
import json

from scipy.io.matlab import loadmat
from scipy.io.matlab import savemat

In [212]:
from sun_utils.build_ds import read_sun_dataset, build_dataset

In [75]:
"""
design draft v3:
{
    image: {
        name
        scene
        layout
        intrinsics
        detector
        filenames: {
            image
            fullres
            depth
        }
        segmentation_arr
        objects : {
            obj_id: {
                label
                globalID
                2D: {
                    polygon: []
                    rectangle: [
                },
                3D: {
                    depth
                },
                phys_props: {
                    
                }
            }
            .
            .
            .
        }
    }
}
"""
None

# Build dataset

In [208]:
all_ds_files = glob.glob('./SUNRGBD/*/*/*/annotation2Dfinal/*') + glob.glob('./SUNRGBD/*/*/*/*/*/annotation2Dfinal/*')

In [1029]:
import pickle

In [1040]:
with open('image_heights.pkl', 'rb') as f:
    img_heights = pickle.load(f)

In [1041]:
len(img_heights)

10270

In [1123]:
sun_ds = read_sun_dataset(all_ds_files)

100%|██████████| 10343/10343 [00:17<00:00, 577.81it/s] 


In [1014]:
sun_ds['img_0063'].keys()

dict_keys(['objects', 'intrinsics', 'scene', 'imgPath'])

In [1028]:
sun_ds['img_0063']['objects']['2D']['frames'][0]['polygon'][0]

{'XYZ': [[0.6463785768, -0.8149276972, 1.68589294],
  [1.114413023, -0.8906619549, 1.849921346],
  [1.300476313, -0.1937646419, 2.118254662],
  [1.120898366, -0.1578154862, 1.973999977],
  [0.9657916427, -0.157877028, 1.974769831],
  [0.9464537501, 0.1455664337, 1.973999977],
  [0.6835758686, 0.2090383172, 2.038234234],
  [0.660431087, -0.3856904507, 2.015401602],
  [0.7216860652, -0.3940743804, 1.854435086],
  [0.6959689856, -0.5536916256, 1.900960922],
  [0.6971981525, -0.5546694994, 1.904318333],
  [0.6785163879, -0.8924766779, 1.824569106]],
 'annotator': '112.95.194.147',
 'object': 1,
 'x': [477, 591, 597, 573, 532, 527, 452, 448, 480, 468, 468, 471],
 'y': [-37, -36, 167, 173, 173, 253, 268, 115, 104, 63, 63, -40]}

In [987]:
pqa_ds_scaff = build_dataset(sun_ds)

# Build from matlab

In [712]:
# updated_2d_bbs = loadmat('./SUNRGBDMeta2DBB_v2.mat', squeeze_me=True)
updated_3d_bbs = loadmat('./SUNRGBDMeta3DBB_v2.mat', squeeze_me=True)['SUNRGBDMeta'].T

In [761]:
updated_3d_bbs[0].dtype

dtype([('sequenceName', 'O'), ('Rtilt', 'O'), ('K', 'O'), ('depthpath', 'O'), ('rgbpath', 'O'), ('anno_extrinsics', 'O'), ('depthname', 'O'), ('rgbname', 'O'), ('sensorType', 'O'), ('valid', 'O'), ('gtCorner3D', 'O'), ('groundtruth3DBB_tight', 'O'), ('groundtruth2DBB_tight', 'O'), ('groundtruth2DBB_full', 'O'), ('groundtruth2DBB_tight_proj', 'O'), ('groundtruth3DBB', 'O')])

In [798]:
updated_3d_bbs['groundtruth3DBB'][0].dtype

dtype([('basis', 'O'), ('coeffs', 'O'), ('centroid', 'O'), ('classname', 'O'), ('sequenceName', 'O'), ('orientation', 'O'), ('label', 'O')])

In [800]:
def build_v2_addition(v2_matlab_arr):
    v2_anno = {}
#     for i in range(v2_matlab_arr.shape[0]):
    for i in range(100):
        try:
            image_name = v2_matlab_arr[i]['sequenceName'].split('/')[-1]
            bboxes = v2_matlab_arr['groundtruth2DBB'][i][0][0]
            v2_anno[image_name] = {
                'objects': {
                    bboxes[j]['objid'].item(): 
                    {
                        'classname': bboxes[j]['classname'].item(),
                        'has3D': bboxes[j]['has3dbox'].item(), 
                        'rect': bboxes[j]['gtBb2D'].tolist()
                    } 
                    for j in range(bboxes.shape[0])
                }
            }
        except IndexError as e:
             v2_anno[image_name] = {}
    return v2_anno

def build_v2_addition_3d(v2_matlab_arr):
    v2_anno = {}
    for i in range(100):
        try:
            image_name = v2_matlab_arr[i]['sequenceName'].split('/')[-1]
            bboxes = v2_matlab_arr[i]['groundtruth3DBB']
#             print(bboxes.dtype.names)
            v2_anno[image_name] = {
                'objects': {
                    j: 
                    {
                        'classname': bboxes['classname'][j],
                    } 
                    for j in range(bboxes.shape[0])
                }
            }
        except IndexError as e:
             v2_anno[image_name] = {}
    return v2_anno

In [801]:
# v2_2d = build_v2_addition(updated_2d_bbs)

In [822]:
v2_3d = build_v2_addition_3d(updated_3d_bbs)

In [351]:
test_array = updated_2d_bbs[:10]

In [340]:
list(v2_2d.values())[0].keys()

dict_keys(['objects'])

In [826]:
sun_objs = []
for image, objs in v2_2d.items():
#     names = ['name','objects']
#     formats = ['O','O']
#     dtype = dict(names = names, formats=formats)
#     object_array = np.array(list(objs['objects'].items()), dtype=dtype)
    image_array = {'objects': list(objs['objects'].items()), 'imageName': image}
    sun_objs.append(image_array)

In [809]:
sun_objs = []
for image, objs in v2_3d.items():
    if not objs:
        continue
    image_array = {'objects': list(objs['objects'].items()), 'imageName': image}
    sun_objs.append(image_array)

In [808]:
object_array.dtype

dtype([('name', 'O'), ('objects', 'O')])

In [371]:
# sample_sun_objs = {k: v for k, v in sun_objs.items() if not v.update({'attached': 1})}

In [814]:
savemat('test_order_3d.mat', {'sun3DMeta': sun_objs})

In [815]:
test_order = loadmat('test_order_3d.mat', squeeze_me=True)

In [819]:
test_order['sun3DMeta'][0]

array((array([[0, array(('bed',), 
      dtype=[('classname', 'O')])],
       [1, array(('night_stand',), 
      dtype=[('classname', 'O')])],
       [2, array(('ottoman',), 
      dtype=[('classname', 'O')])],
       [3, array(('dresser_mirror',), 
      dtype=[('classname', 'O')])],
       [4, array(('dresser',), 
      dtype=[('classname', 'O')])],
       [5, array(('lamp',), 
      dtype=[('classname', 'O')])]], dtype=object), '000002_2014-05-26_14-23-37_260595134347_rgbf000103-resize'), 
      dtype=[('objects', 'O'), ('imageName', 'O')])

In [699]:
test_order['sun2DMeta'][0]

array((array([[0,
        array(('bed', 1, array([328, 152, 346, 320]), 1), 
      dtype=[('classname', 'O'), ('has3D', 'O'), ('rect', 'O'), ('attached', 'O')])],
       [1,
        array(('night_stand', 1, array([ 572.54180602,  275.63545151,   98.37792642,   53.17725753]), 1), 
      dtype=[('classname', 'O'), ('has3D', 'O'), ('rect', 'O'), ('attached', 'O')])],
       [2,
        array(('ottoman', 1, array([ 332.35785953,  318.17725753,  316.40468227,  209.1638796 ]), 1), 
      dtype=[('classname', 'O'), ('has3D', 'O'), ('rect', 'O'), ('attached', 'O')])],
       [3,
        array(('dresser_mirror', 1, array([ 157.75919732,   10.63545151,  156.8729097 ,  362.4916388 ]), 1), 
      dtype=[('classname', 'O'), ('has3D', 'O'), ('rect', 'O'), ('attached', 'O')])],
       [4,
        array(('dresser', 1, array([  1,   1, 221, 529]), 1), 
      dtype=[('classname', 'O'), ('has3D', 'O'), ('rect', 'O'), ('attached', 'O')])],
       [5,
        array(('lamp', 0, array([ 614.19732441,  200.30

In [507]:
test_order['sun2DMeta'].T[0][0][0][0][0][0][0]

array([[0]])

In [512]:
iname, obs = list(v2_2d.items())[0]

In [634]:
all_objects = updated_2d_bbs['SUNRGBDMeta2DBB']['groundtruth2DBB']

In [652]:
range(all_objects.shape[0])

range(0, 10335)

In [659]:
all_objects[100]['classname']

array(['printer', 'counter', 'cabinet', 'door'], dtype=object)

In [686]:
all_obj_list = [all_objects[i]['classname'].tolist() for i in range(all_objects.shape[0]) if all_objects[i].dtype]

In [690]:
all_obj_list_flat = flattened_list = [item for sublist in all_obj_list for item in sublist]

In [696]:
len((all_obj_list_flat))

81672

# Write annotation

In [1155]:
import os

In [846]:
_ = '''
{
  "text": {
    "T40": {
      "category": "unlabeled",
      "box_id": "T40",
      "source": {
        "additionalProperties": false,
        "$schema": "http://json-schema.org/draft-04/schema",
        "type": "object",
        "properties": [
          {
            "book_source": "Confusing_Science_Terms_Grades_5_-_8_Schyrlet_Cameron"
          },
          {
            "page_n": 12
          }
        ]
      },
      "score": 1,
      "contents": "BATTER",
      "v_dim": 2153,
      "rectangle": [
        [
          788,
          840
        ],
        [
          862,
          856
        ]
      ]
    },
'''

In [1105]:
len(sun_ds['img_0162']['objects']['2D']['objects'])

13

In [1092]:
# sun_ds['img_0063']

In [1110]:
sun_ds['img_0162']['objects']['2D']['frames'][0]['polygon']

[{'annotator': '112.95.194.147',
  'object': 6,
  'x': [300,
   195,
   -41,
   -39,
   -8,
   0,
   283,
   285,
   239,
   240,
   255,
   263,
   262,
   271,
   279,
   279,
   302],
  'y': [11,
   -46,
   -43,
   156,
   381,
   432,
   434,
   348,
   340,
   328,
   303,
   282,
   239,
   236,
   235,
   39,
   37]},
 {'object': 0,
  'x': [270, 284, 298, 308, 290, 274, 262, 261],
  'y': [326, 328, 325, 245, 240, 238, 244, 251]},
 {'object': 1,
  'x': [303, 314, 325, 331, 335, 345, 346, 342, 337, 337, 325, 308],
  'y': [303, 309, 308, 302, 295, 288, 279, 273, 272, 265, 259, 259]},
 {'object': 1, 'x': [335, 333, 339], 'y': [277, 286, 282]},
 {'object': 2,
  'x': [322, 320, 294, 248, 263, 267, 278, 292, 300, 302, 311],
  'y': [312, 325, 334, 331, 287, 325, 329, 329, 325, 307, 310]},
 {'object': 3, 'x': [414, 450, 463, 463, 410], 'y': [325, 317, 314, 310, 320]},
 {'object': 4,
  'x': [429, 476, 488, 484, 435, 435, 481, 416, 416],
  'y': [300, 306, 306, 273, 249, 275, 300, 295, 298]

In [1107]:
sorted(sun_ds['img_0162']['objects']['2D']['frames'][0]['polygon'], key=lambda x: x['object'])

[{'object': 0,
  'x': [270, 284, 298, 308, 290, 274, 262, 261],
  'y': [326, 328, 325, 245, 240, 238, 244, 251]},
 {'object': 1,
  'x': [303, 314, 325, 331, 335, 345, 346, 342, 337, 337, 325, 308],
  'y': [303, 309, 308, 302, 295, 288, 279, 273, 272, 265, 259, 259]},
 {'object': 1, 'x': [335, 333, 339], 'y': [277, 286, 282]},
 {'object': 2,
  'x': [322, 320, 294, 248, 263, 267, 278, 292, 300, 302, 311],
  'y': [312, 325, 334, 331, 287, 325, 329, 329, 325, 307, 310]},
 {'object': 2,
  'x': [589, 588, 581, 591, 599, 599],
  'y': [130, 152, 172, 177, 177, 129]},
 {'object': 2,
  'x': [597, 572, 562, 597, 598, 589, 580, 586, 586, 599, 599],
  'y': [121, 118, 234, 238, 180, 178, 173, 152, 129, 127, 125]},
 {'object': 3, 'x': [414, 450, 463, 463, 410], 'y': [325, 317, 314, 310, 320]},
 {'object': 4,
  'x': [429, 476, 488, 484, 435, 435, 481, 416, 416],
  'y': [300, 306, 306, 273, 249, 275, 300, 295, 298]},
 {'object': 4, 'x': [430, 428, 469, 432], 'y': [263, 292, 296, 277]},
 {'object': 4,
 

In [1446]:
def make_annotation_json(image_name, image_annotation):
    def get_bbox(polygon):
        xs = [max(0, x) for x in polygon['x']]
        ys = [max(0, y) for y in polygon['y']]
        return [[min(xs), min(ys)], [max(xs), max(ys)]]
    
    image_objects = image_annotation['objects']['2D']['objects']
    image_poly = sorted(image_annotation['objects']['2D']['frames'][0]['polygon'], key=lambda x: x['object'])
    image_path = image_annotation['imgPath']
    annotations = {}
    for obj_id in range(len(image_poly)):
        box_id = 'T' + str(obj_id)
        annotations[box_id] = {
            "box_id": box_id,
            "category": 'unlabeled',
            "rectangle": get_bbox(image_poly[obj_id]),
            "v_dim": img_heights[image_name],
            "contents": image_objects[image_poly[obj_id]['object']]['name']
        }
#     with open('image_annotation_scaff/' + image_name + '.json', 'w') as f:
#         json.dump({'text': annotations}, f, indent=5)
    return {image_name: image_path + '/' + image_name}, {'text': annotations}

In [1140]:
sampled_images = np.random.choice(list(sun_ds.keys()), 500)

In [1401]:
remaining_images = [img for img in sun_ds.keys() if img not in sampled_images]

In [1537]:
len(remaining_images)

9782

In [1416]:
with open('rem_image.pkl', 'wb') as f:
    pickle.dump(remaining_images, f, protocol=2)

In [1452]:
all_errors = []
image_paths = {}
sample_annotations = {}
for image_name in list(remaining_images) + list(sampled_images):
    try:
        annotations = sun_ds[image_name]
        ipath, anno = make_annotation_json(image_name, annotations)
        image_paths.update(ipath)
        sample_annotations.update({image_name: anno})
    except (IndexError, ValueError, KeyError, TypeError) as e:
        all_errors.append(e)
        continue

In [1453]:
len(image_paths)

9606

In [1195]:
all_image_files = glob.glob('./SUNRGBD/*/*/*/image/*')

In [1412]:
for image_name, i_path in image_paths.items():
    original_image_dir = os.path.join(i_path, 'image')
    old_name = os.listdir(original_image_dir)[0]
    original_image_path = os.path.join('/Users/schwenk/wrk/pqa/sunrgbd', i_path, 'image', old_name)
    base_image_path = '/Users/schwenk/wrk/pqa/ai2-vision-turk-data/pqa-annotation/images/'
    new_image_path = base_image_path + image_name + '.jpg'
    os.symlink(original_image_path, new_image_path)

# Visualize annotation results

In [1244]:
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw

In [1707]:
cons_res_df_1 = pd.read_pickle('../../pqa/turk_processing/first_round_results.pkl')
cons_res_df_2 = pd.read_pickle('../../pqa/turk_processing/second_round_results.pkl')
cons_res_df_3 = pd.read_pickle('../../pqa/turk_processing/filtered_second_round_results.pkl')
cons_res_df_4 = pd.read_pickle('../../pqa/turk_processing/third_round_results.pkl')
cons_res_df_5 = pd.read_pickle('../../pqa/turk_processing/combined_round2_3_results.pkl')
cons_res_df_6 = pd.read_pickle('../../pqa/turk_processing/round_4_qual.pkl')

In [1708]:
# cons_res_df = cons_res_df_1.append(cons_res_df_2)
cons_res_df = cons_res_df_6

In [1709]:
cons_res_df['image_name'] = cons_res_df['page'].apply(lambda x: x.split('.jpg')[0])

In [1710]:
attached_item_count = cons_res_df[cons_res_df['category'] == 'Attached'].shape[0]
attached_item_count / cons_res_df.shape[0]

0.1391144693171386

In [1711]:
n_images_w_attached = pd.unique(cons_res_df[cons_res_df['category'] == 'Attached']['page']).shape[0]
n_images_w_attached / pd.unique(cons_res_df['page']).shape[0]

0.8345021037868162

In [1712]:
def lookup_box_rect(row):
    part_anno = sample_annotations[row['page'].split('.jpg')[0]]
    rect = part_anno['text'][row['box_id']]['rectangle']
    return tuple(rect[0]), tuple(rect[1])

In [1713]:
cons_res_df['rect'] = cons_res_df.apply(lookup_box_rect, axis=1)
# cons_res_df_na = cons_res_df[~cons_res_df['page'].isin(pages_with_attached)]
cons_res_attached = cons_res_df[cons_res_df['category'] == 'Attached']

In [1714]:
grouped_cons_by_page = cons_res_attached.groupby('page')
page_agged = grouped_cons_by_page.agg(lambda x: tuple(x))

In [1715]:
pages_with_attached = page_agged.index.tolist()

In [1716]:
def draw_attached(image_name, boxes):
    image_dir_name = image_name.split('.jpg')[0]
    image_path = glob.glob(os.path.join(image_paths[image_dir_name], 'image', '*' + '.jpg'))[0]
    pimage = Image.open(image_path)
    for rect in boxes:
        draw = ImageDraw.Draw(pimage)
        draw.rectangle(rect)
    pimage.save('./new_attached_task_renders/' + image_name)

In [1717]:
samples_to_draw = page_agged.sample(100)

In [1718]:
for i in range(samples_to_draw.shape[0]):
    draw_attached(samples_to_draw.index[i], samples_to_draw['rect'][i])

# match to 3D annotations

In [1538]:
def area_intersect_over_union(segmentImage1, segmentImage2, segmentArea1 = 0, segmentArea2 = 0):
    areaIntersection = np.sum(segmentImage1 & segmentImage2)
    if areaIntersection == 0:
        return 0, 0, 0
    if segmentArea1 == 0:
        segmentArea1 = np.sum(segmentImage1)
    if segmentArea2 == 0:
        segmentArea2 = np.sum(segmentImage2)
    areaOverlap = areaIntersection / (segmentArea1 + segmentArea2 - areaIntersection + 0.01)
    areaSelfOverlap1 = areaIntersection / (segmentArea1 + 0.01)
    areaSelfOverlap2 = areaIntersection / (segmentArea2 + 0.01)
    return areaOverlap, areaSelfOverlap1, areaSelfOverlap2


def comp_iou(raw_image, ocr_box, anno_box):

    def update_img(mask, box):
        # box
        # some rectangle of the structure [[start_x, start_y], [end_x, end_y]
        start_x, start_y = box[0]
        end_x, end_y = box[1]
        mask[start_y:end_y + 1, start_x:end_x + 1] = True

    mask_a = np.zeros(raw_image.shape[0:2], dtype=np.bool)
    mask_b = np.zeros(raw_image.shape[0:2], dtype=np.bool)
    update_img(mask_a, ocr_box['rectangle'])
    update_img(mask_b, anno_box['rectangle'])

    areaOverlap, areaSelfOverlap1, areaSelfOverlap2 = areaIntersectOverUnion(mask_a, mask_b)

    #iou = (mask == 2).sum() / (mask >= 1).sum()
    return areaOverlap, areaSelfOverlap1, areaSelfOverlap2 

In [1677]:
grouped_by_page = cons_res_df_5.groupby('page')

In [1686]:
single_test_image_df = grouped_by_page.get_group(cons_res_df_5['page'][500])

In [1693]:
test_image_name = single_test_image_df['page'].iloc[0].replace('.jpg', '')

In [1696]:
# sun_ds['img_0162']['objects']['2D']['frames'][0]['polygon']

In [1698]:
sun_ds[test_image_name]['objects'].keys()

dict_keys(['2D'])

In [1704]:
list(sun_ds_scaf.values())[0]['objects']['3D']

{'conflictList': [None],
 'date': 'Wed, 04 Mar 2015 02:38:57 GMT',
 'extrinsics': '20140915210114.txt',
 'fileList': ['img_0063.jpg'],
 'frames': [{}],
 'name': 'b3do/img_0063//',
 'objects': [{'name': 'bookshelf',
   'polygon': [{'X': [-0.15,
      0.6968309773606108,
      0.5599692604758523,
      -0.2868617168847586],
     'Ymax': 1.0772727272727274,
     'Ymin': -0.6590909090909091,
     'Z': [1.7454545454545454,
      2.0797299312547852,
      2.4264462806961755,
      2.0921708948959292],
     'rectangle': True}]},
  None,
  {'name': 'bookshelf',
   'polygon': [{'X': [-0.22311008247374234,
      -1.0863636363636364,
      -1.2424112173688295,
      -0.3791576634789356],
     'Ymax': 1.0823231090198864,
     'Ymin': -0.6590909090909091,
     'Z': [1.6964473554359838,
      1.3727272727272728,
      1.7888541554077861,
      2.112574238116498],
     'rectangle': True}]},
  None,
  {'name': 'chair',
   'polygon': [{'X': [0.46866039115813524,
      0.6627761060708446,
      1.367407

# Hide

In [146]:
# %page single_image_anno

In [ ]:
# updated_2d_bbs = loadmat('./SUNRGBDMeta2DBB_v2.mat')['SUNRGBDMeta2DBB'].T
# updated_3d_bbs = loadmat('./SUNRGBDMeta3DBB_v2.mat')['SUNRGBDMeta'].T
# v2_suppl_2d = build_v2_addition(updated_2d_bbs)

In [71]:
# fnl = [anno['2D']['fileList'] for anno in sun_ds_scaf.values() if '2D' in anno.keys() and len(anno['2D']['fileList']) == 1]

# pd.Series([len(anno['2D']['fileList']) for anno in sun_ds_scaf.values() if '2D' in anno.keys()]).value_counts()

# cflct = [anno['2D']['conflictList'] for anno in sun_ds_scaf.values() if '2D' in anno.keys() and len(anno['2D']['fileList']) == 1]